In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from glob import glob
from configs import *
from tqdm import tqdm
from utils import *
import numpy as np
import matplotlib.pyplot as plt
from dataset import *
from time import time
tqdm.monitor_interval = 0
import torch
import json
import torch.nn as nn
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

In [3]:
LAG = 12
EPS = 30
SIND = 30
HSIZE = 256
DEEP = False
SROUTE = SAMPLE_ROUTES[0]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
dset = SingleStop(SROUTE, SIND, 'train', 32, lag=LAG).generator()
evalset = SingleStop(SROUTE, SIND, 'test', 32, lag=LAG).generator(shuffle=False)

Locals dataset: train
 [*] Loaded routes: 1 (0.33s)
 [*] Has trainable inds: 262262
 [*] Subset train: 212106
 [*] Subset in Stop-30: 6079
Locals dataset: test
 [*] Loaded routes: 1 (0.35s)
 [*] Has trainable inds: 262262
 [*] Subset test: 50156
 [*] Subset in Stop-30: 1549


In [9]:
from models.temporal.RNN import RNN

In [12]:
model = RNN(hidden_size=HSIZE, deep=DEEP, lag=LAG, steps=1).to(device)
model.device = device

In [13]:
criterion, opt, sch = model.params(lr=0.001)

In [14]:
evaluate(evalset, model, crit=lambda y, _y: mape(tonpy(y)[:, 0], tonpy(_y)[:, 0]))
evaluate(evalset, model, crit=lambda y, _y: criterion(y[:, 0], _y[:, 0]).item())

Eval loss: 91.2241   
Eval loss: 12.8127   


In [ ]:
# evaluate(evalset, model, crit=lambda _y, y: mape(tonpy(_y), tonpy(y), yavg))
# evaluate(evalset, model, crit=lambda _y, y: criterion(_y, y).item())

In [15]:
losses = []
eval_mse = []
eval_mape = []
from numpy.random import randint
for eii  in range(EPS):
    for bii, batch in enumerate(dset):
        model.train()
#         indend = 2 + randint(10)
        indend = 12
        Xs, Ys = model.format_batch(batch[:, :indend])
        
        outputs = model(Xs)

        opt.zero_grad()
        loss = criterion(outputs, Ys)
        loss.backward()
        losses.append(loss.item())
        opt.step()

        sys.stdout.write('[%d/%d : %d/%d] - L%.2f      \r' % (
            eii+1, EPS,
            bii+1, len(dset), 
            loss.item()
        ))
    sys.stdout.write('\n')

    eval_mse.append(
        evaluate(
            evalset, model, 
            crit=lambda y, _y: criterion(y[:, 0], _y[:, 0]).item(), result=True))
    eval_mape.append(
        evaluate(
            evalset, model, 
            crit=lambda y, _y: mape(tonpy(y)[:, 0], tonpy(_y)[:, 0]), result=True))
    sys.stdout.flush()
#     sch.step()

[1/30 : 189/189] - L2.74       
Eval loss: 3.1147    
Eval loss: 46.9135   
[2/30 : 189/189] - L1.54       
Eval loss: 3.0825    
Eval loss: 48.4947   
[3/30 : 189/189] - L31.58      
Eval loss: 3.0530    
Eval loss: 47.5794   
[4/30 : 189/189] - L2.29       
Eval loss: 3.0198    
Eval loss: 46.5780   
[5/30 : 189/189] - L1.90       
Eval loss: 3.0127    
Eval loss: 46.7888   
[6/30 : 189/189] - L1.95       
Eval loss: 3.0163    
Eval loss: 45.2000   
[7/30 : 189/189] - L1.92       
Eval loss: 3.0363    
Eval loss: 49.2879   
[8/30 : 189/189] - L9.83       
Eval loss: 3.0586    
Eval loss: 50.9072   
[9/30 : 189/189] - L6.56       
Eval loss: 3.0139    
Eval loss: 49.6369   
[10/30 : 189/189] - L1.20       
Eval loss: 2.9764    
Eval loss: 44.7533   
[11/30 : 189/189] - L1.50       
Eval loss: 2.9995    
Eval loss: 43.3657   
[12/30 : 189/189] - L2.24       
Eval loss: 3.0015    
Eval loss: 45.4553   
[13/30 : 189/189] - L13.44      
Eval loss: 2.9702    
Eval loss: 47.6348   
[14/30 :

In [17]:
name = 'n10-%s-%d'% (SROUTE, SIND)
torch.save(model.state_dict(), 'checkpoints/rnn/%s.pth' % name)

In [18]:
print(name)

n10-M102_0-30


In [19]:
# plt.figure(figsize=(14, 4))
# plt.subplot(1, 2, 1)
# plt.plot(losses)
# plt.subplot(1, 2, 2)
# plt.plot(eval_mse)
# plt.show();plt.close()

In [29]:
kcast = 5
model.eval()
klosses = [list() for _ in range(kcast)]
for batch in evalset:
    Xs, Ys = model.format_batch(batch[:, :6, :])
    yinit, hidden, yall = model(Xs, dump=True)
    ycasts = [yinit.detach()] # k=1
    xnext = [yinit.detach()]
    for ki in range(1, kcast):
        ynext, hidden, _ = model(xnext, hidden=hidden, dump=True)
        ycasts.append(ynext)
        xnext = [ynext.detach()]

    Xs, _ = model.format_batch(batch[:, 6:, :])
    ypred, ytrue = torch.stack(ycasts, dim=1), torch.stack(Xs[:kcast], dim=1)
    for ki in range(kcast):
        kmse = criterion(ypred[:, ki, 0], ytrue[:, ki, 0]).item()
        kmape = mape(tonpy(ypred[:, ki, 0]), tonpy(ytrue[:, ki, 0]))
        klosses[ki].append([kmse, kmape])
#     bi = 0
#     plt.figure(figsize=(14, 3))
#     plt.plot(tonpy(batch[bi, -2, :]), color='#DDDDDD')
# #     for yent in yall:
# #         print(yent.size())
# #         plt.plot(tonpy(yent[bi]), color='#CCCCCC')
# #     for ki in range(kcast):
#     plt.plot(range(5, 10), tonpy(ypred)[bi])
#     plt.show(); plt.close()
#     assert False
fcast = []
for ki in range(kcast):
    kmse, kmape = zip(*klosses[ki])
    kmse = np.mean(kmse)
    kmape = np.mean(kmape)
    fcast.append(dict(mse=kmse, mape=kmape))
    
# print(kmse, kmape)

with open('checkpoints/rnn/%s.json' % name, 'w') as fl:
    json.dump(dict(
        mse=eval_mse,
        mape=eval_mape,
        fcast=fcast,
    ), fl, indent=4)

#     print(kmse)
#     print(kmape)
#     break
        
#     for ki 
        
#     bi = 0
#     plt.figure(figsize=(14, 3))
#     plt.plot(tonpy(batch[bi, -2, :]), color='#DDDDDD')
#     for yent in yall:
# #         print(yent.size())
#         plt.plot(tonpy(yent[bi]), color='#CCCCCC')
#     for ki in range(kcast):
#         plt.plot(ycasts[ki][bi])
#     plt.show(); plt.close()
#     assert False
        

In [ ]:
# data = history_byname(SROUTE)

# sample = data[int(TSTEPS*0.8):]
# inds, integs = high_integ(sample[LAG:])
# inds += LAG
# # print(inds[0])
# print('Visualize period', inds[0])
# sample = sample[inds[0]-LAG:inds[0]]
# # sample = hist_smooth(sample)
# print(np.count_nonzero(np.isnan(sample)))
# for jj, ii in zip(*np.where(np.isnan(sample))):
#     sample[jj, ii] = sample[jj-1, ii]
# show_context(sample)

In [ ]:
# bi = 30
# hist = torch.Tensor(np.expand_dims(sample[:, bi-10:bi], 0))
# Xs, _ = model.format_batch(hist)
# yout = tonpy(model(Xs))
# print(yout.shape)
# #     preds.append(tonpy(yout))

In [ ]:
# l = show_context(sample, draw=False)
# ylast = np.flip(yout[0, :])
# xpos = list(range(bi-10, bi))
# # plt.plot([25, 25], [-5, 1], color='#666666')
# # plt.plot([25, 25], [9, 7], color='#666666')
# p = plt.plot(xpos, ylast, color='C1')
# plt.xlim(19, 31)
# plt.ylim(0, 8)
# plt.legend([l[0], p[0]], ['Known Travel Speed', 'Predicted Travel Speed'])
# plt.show(); plt.close()